In [ ]:
# CÉLULA 1: Enviar o arquivo kaggle.json para o Colab
# Ao rodar esta célula, um botão de "Escolher arquivos" vai aparecer.
# Selecione o arquivo 'kaggle.json' que você baixou do Kaggle.

from google.colab import files

print("Por favor, faça o upload do seu arquivo kaggle.json")
files.upload()

# ---
# Agora, execute a próxima célula para configurar a API
# ---

Por favor, faça o upload do seu arquivo kaggle.json


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"victorquerinomartins","key":"84c209d856fab875aecad8d30d1e4f76"}'}

In [ ]:
# CÉLULA 2: Configurar a API do Kaggle no ambiente do Colab
# Este código move o arquivo json para o lugar certo para que a API funcione.

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("API do Kaggle configurada com sucesso!")

# ---
# Agora, execute a próxima célula para baixar o dataset
# ---

API do Kaggle configurada com sucesso!


In [ ]:
# CÉLULA 3: Baixar o dataset de veículos do Kaggle
# Usaremos um dataset bem conhecido e estruturado.
# O comando !kaggle ... é como se estivéssemos digitando no terminal.

!kaggle datasets download -d lyensoetanto/vehicle-images-dataset

print("\nDataset baixado. Agora vamos descompactar.")

# ---
# Agora, execute a próxima célula para descompactar o arquivo
# ---

Dataset URL: https://www.kaggle.com/datasets/lyensoetanto/vehicle-images-dataset
License(s): unknown
  0% 0.00/80.7M [00:00<?, ?B/s]
100% 80.7M/80.7M [00:00<00:00, 1.20GB/s]

Dataset baixado. Agora vamos descompactar.


In [ ]:
# CÉLULA 4: Descompactar o arquivo .zip
# O arquivo baixado precisa ser extraído para podermos usar as imagens.

import zipfile

with zipfile.ZipFile('vehicle-images-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('vehicle_dataset')

print("Dataset descompactado na pasta 'vehicle_dataset'")

# ---
# Finalmente, execute a última célula com o código de treinamento!
# ---

Dataset descompactado na pasta 'vehicle_dataset'


In [ ]:
# CÉLULA 5: Código de Treinamento da Rede Neural (ajustado para as novas pastas)
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Listar o conteúdo da pasta descompactada para encontrar o caminho correto
print("Conteúdo da pasta 'vehicle_dataset':")
!ls vehicle_dataset

# 1. Apontar para a pasta do dataset que acabamos de descompactar
# A estrutura deste dataset é um pouco diferente: Vehicle Image Dataset/DATA/CAR
# base_dir = os.path.join('vehicle_dataset', 'Vehicle Image Dataset', 'DATA')

# **Ajuste este caminho após verificar a estrutura da pasta acima!**
# Exemplo: base_dir = os.path.join('vehicle_dataset', 'caminho_correto_para_imagens')
base_dir = 'vehicle_dataset' # Placeholder - you will need to set this correctly

# 2. Criar datasets de treino e validação
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# Usando 80% para treino e 20% para validação
train_dir = base_dir # Vamos tentar usar base_dir e deixar image_dataset_from_directory encontrar as classes
validation_dir = base_dir

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print("Classes encontradas:", class_names)

# O restante do código é o mesmo que já conhecemos...

# Otimizar dataset para desempenho
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Criar modelo
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Treinar modelo
history = model.fit(train_ds, validation_data=test_ds, epochs=15)

# Matriz de confusão
y_true = []
y_pred = []
for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title("Matriz de Confusão - Kaggle Veículos")
plt.show()

Conteúdo da pasta 'vehicle_dataset':
'Big Truck'  'Multi Purpose Vehicle'  'Sport Utility Vehicle'   Van
'City Car'    Sedan		       Truck
Found 15645 files belonging to 7 classes.
Using 12516 files for training.
Found 15645 files belonging to 7 classes.
Using 3129 files for validation.
Classes encontradas: ['Big Truck', 'City Car', 'Multi Purpose Vehicle', 'Sedan', 'Sport Utility Vehicle', 'Truck', 'Van']
Epoch 1/15


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


392/392 ━━━━━━━━━━━━━━━━━━━━ 648s 2s/step - accuracy: 0.3599 - loss: 1.6269 - val_accuracy: 0.5270 - val_loss: 1.1516
Epoch 2/15
392/392 ━━━━━━━━━━━━━━━━━━━━ 638s 2s/step - accuracy: 0.6063 - loss: 1.0098 - val_accuracy: 0.6676 - val_loss: 0.8454
Epoch 3/15
392/392 ━━━━━━━━━━━━━━━━━━━━ 608s 2s/step - accuracy: 0.7389 - loss: 0.6724 - val_accuracy: 0.7021 - val_loss: 0.7761
Epoch 4/15
392/392 ━━━━━━━━━━━━━━━━━━━━ 617s 2s/step - accuracy: 0.8366 - loss: 0.4455 - val_accuracy: 0.7737 - val_loss: 0.5992
Epoch 5/15
176/392 ━━━━━━━━━━━━━━━━━━━━ 5:14 1s/step - accuracy: 0.9020 - loss: 0.2717

KeyboardInterrupt: 